# Lab Besos Work Flow from idf to surrogate desgin optimization


In this laboratory you will go over some of the basic work flow to create a a surrogate design optimization model from an EnergyPlus simulation. You will train a surrogate model network to find optimal design parameters.

In [ ]:
#!pip install besos --user
%matplotlib inline
import time
import numpy as np
import pandas as pd


from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP,EvaluatorGeneric
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem, Problem
from besos import eppy_funcs as ef, sampling

import matplotlib.pyplot as plt
from seaborn import heatmap
from seaborn import pairplot
from plotly import express as px
import plotly

In [ ]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Building presentation

In this Laboratory you will have to perform a surrogate optimization of a sample residential building unit. The considered unit is composed by a typical flat of a multi-storey building. Its internal organisation and definition is in line with suggested residential building typologies included in well-known architectural technical manuals  The considered building is composed by two units for floor, while a single unit is simulated for this chapter. The simulated spaces are considered to be at an intermediate floor with an upper floor and lower floor working at the same temperature (adiabatic). Similarly, the simulated unit is touching a specular one: confining walls are also assumed as adiabatic. Upper-floor balconies are included to consider shading effects.

![Image](image/building_lab.png)

## Load the Building 

The building is defined by the Information Data File (IDF) or using the new EnergyPlus format (IDF).

In [ ]:
# Open the IDF file
building = ef.get_building('lab_building.idf')
building.view_model()

## Evaluator
## Set up the inputs and outputs of your exploration

- what properties of the building will we be changing?
- what are some of the performance metrics of the building that we want to explore?
- what external weather conditions is the building experiencing?


![Image](image/setting_up_the_evaluator.PNG)

### Load and Display Weather Data

In [ ]:
#The weather conditions are specified in the EnergyPlus Weather File (EWP) file. The properties we will change in the building will be defined in the parameter space. In the objectives we will specify the what output performance metrics we wish to extract such that we can explore them later.

In [ ]:
from epw import epw
meteo = epw()
    

In [ ]:
epw_file='PARIS_FR-hour.epw'
meteo.read(epw_file)


In [ ]:
meteo.dataframe


In [ ]:
meteo.dataframe.describe()

# Define The Problem

1. Define the parameters and your objectives you whant to change
2. Create Selectors for getting the paramters fileds in the model
3. Describe the parameter variation
4. Define your problem

In [ ]:
building.idfobjects

In [ ]:
roof_ins = FieldSelector(class_name='Material',
                         object_name='MW Glass Wool (rolls)_.1445',
                         field_name='Thickness')
wall_ins = FieldSelector(class_name='Material', 
                         object_name='EPS Expanded Polystyrene (Heavyweight)_.1', 
                         field_name='Thickness')

In [ ]:
wall_range = RangeParameter(min_val=0.1,max_val=0.5)
roof_range = RangeParameter(min_val=0.1,max_val=0.8)


We can combine this with the `Selector` above to get a `Parameter`:

In [ ]:
insulation_param = [Parameter(selector=wall_ins,
                                 value_descriptor=wall_range ,
                                 name='Wall Insulation'),
                   Parameter(selector=roof_ins,
                                 value_descriptor=roof_range ,
                                 name='Roof Insulation')]
print(insulation_param)


In [ ]:
[window for window in building.idfobjects['FENESTRATIONSURFACE:DETAILED'] if window.Surface_Type=='Window']

In [ ]:
windows = FieldSelector(class_name='FenestrationSurface:Detailed', 
                        object_name='*',
                        field_name='Construction Name')

In [ ]:
windows.get_objects(building)

In [ ]:
#selection of windows parameters
win_arr = ['single_glazing','double_glazing','triple_glazing']
windowsTypes = CategoryParameter(win_arr)

windowsParameters = []

for fenestration in building.idfobjects["FenestrationSurface:Detailed"]:
     if fenestration.obj[2] == "Window":
        sel = FieldSelector(class_name ='FenestrationSurface:Detailed', 
                            object_name = fenestration.Name, 
                            field_name='Construction Name' )
        windowsParameters.append(Parameter(selector=sel, 
                                           value_descriptors = windowsTypes, 
                                           name='Windows types'))
        

In [ ]:
windowsParameters

In [ ]:
#ACH parameters selection
ventAchRange = RangeParameter(min_val = 0.0, max_val=6.0)

ventilationAchParam = []


ventACH_sel = FieldSelector(class_name = 'ZoneVentilation:DesignFlowRate', 
                    object_name = '*', 
                    field_name = 'Air Changes per Hour')
ventilationAchParam.append(Parameter(selector=ventACH_sel, 
                                     value_descriptors=ventAchRange, 
                                     name='Ventilation ACH'))


In [ ]:
ventilationAchParam

In [ ]:
lights_selector = FieldSelector(class_name='Lights', object_name='*', field_name='Watts per Zone Floor Area')
lights_range = RangeParameter(min_val=8,max_val=20)

In [ ]:
lights_param =     Parameter(
        lights_selector,
        value_descriptor=lights_range ,
        name="Lights Watts/Area",
    )

In [ ]:
[shadeControl for shadeControl in building.idfobjects["WINDOWSHADINGCONTROL"]]

In [ ]:
TempShadingRange = RangeParameter(min_val = 18, max_val=30)
RadShadingRange = RangeParameter(min_val = 80, max_val=300)

setpointParams = []
shade_setpoint_sel = FieldSelector(class_name ='WindowShadingControl', 
                    object_name = '*', 
                    field_name='Setpoint' )
shade_setpoint2_sel = FieldSelector(class_name ='WindowShadingControl', 
                     object_name = '*', 
                     field_name='Setpoint 2' )
setpointParams.append(Parameter(selector=shade_setpoint_sel, value_descriptor = TempShadingRange, name='Temp Setpoint shading'))
setpointParams.append(Parameter(selector=shade_setpoint2_sel, value_descriptor = RadShadingRange, name='Rad Setpoint shading'))

In [ ]:
parameters = insulation_param + windowsParameters + [lights_param] + ventilationAchParam + setpointParams

In [ ]:
parameters

In [ ]:
objectives = ['Electricity:Facility','DistrictHeating:Facility','DistrictCooling:Facility'] # these get made into `MeterReader` or `VariableReader`

problem=EPProblem(parameters, objectives) # problem = parameters + objectives



In [ ]:
problem.names()

## Generate the Dataset

1. Generate 10 samples with sampling strategy
2. Setup the parallel processing
3. Simulate the Samples
4. Store and recover the expensive runs

In [ ]:
samples = sampling.dist_sampler(sampling.lhs, problem, num_samples=40)
samples

In [ ]:
evaluator = EvaluatorEP(problem, 
                        building, 
                        out_dir='outputdir', 
                        err_dir='outputdir',
                        epw_file=epw_file,
                        progress_bar=True)



Run the samples and calculate the execution time

In [ ]:
t1 = time.time()
# Run Energyplus
sim_samples = evaluator.df_apply(samples,
                             keep_input=True, 
                             #keep_dirs=True, 
                             processes=4)  # flag keep_dirs to True to save all ouput
t2 = time.time()
time_of_sim = t2 - t1

Calculate the time

In [ ]:
def niceformat(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return hour, minutes, seconds


hours, mins, secs = niceformat(time_of_sim)

print(
    "The total running time: {:2.0f} hours {:2.0f} min {:2.0f} seconds".format(
        hours, mins, secs
    )
)
# Build a results DataFrame

## Store the expensive calculations

Since this can quite a big run. Lets store the results such that we don't have to rerun this problem.

In [ ]:

sim_samples.to_pickle("simulation_sample_40.pkl")

## Analize and describe your simulation output

In [ ]:
sim_samples.describe()

In [ ]:
sim_samples

In [ ]:
sim_samples = sim_samples.sort_values(by=objectives[1])
ax=sim_samples.plot.bar(subplots=True,legend=None, figsize=(12,20))

## Visualising the parametric analysis

A better way to analyse the results is by looking at scatter plots of the inputs versus the outputs.  
This enables us to visually see strong relationships of inputs and outputs.

In [ ]:
plt.rcParams.update({'font.size': 18})
_=pairplot(sim_samples,x_vars=samples.columns, y_vars=objectives, kind="scatter",height=4)

### Correlation heat map
Another way to analyse the impact of the inputs on the outputs is by analysing the correlation.  
A common metric is the Pearsson correlation coefficient:

$ r = \frac{N\sum{XY}-(\sum{X}\sum{Y})}{\sqrt{ [N\sum{x^2}-(\sum{x})^2 ][N\sum{y^2}-(\sum{y})^2 }]} $

where N is the number of samples. $X$ is the vector of observation of variable 1 (e.g. wall conductivity) and $Y$ is the vetor of observations of variable 2 (e.g. electricity consumption).  
The closer $r$ is to one the stronger the correlation, and similarly for negative one and negative correleation.

To visualize the correlation coefficients of all inputs and outputs, we can plot a heatmap:

In [ ]:
corr=sim_samples.corr()
corr

In [ ]:
corr.drop(objectives, axis = 1, inplace = True)
corr.drop(['Roof Insulation','Wall Insulation', 'Lights Watts/Area','Ventilation ACH','Temp Setpoint shading', 'Rad Setpoint shading'], axis = 0 ,inplace = True)
corr

In [ ]:
plt.rcParams.update({'font.size': 14})
plt.figure(figsize = (13,10))
_ = heatmap(corr,annot=True)


# Setup the dataset for the Surrogate Model

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import warnings

In [ ]:
features = list(samples.columns)
print(features)
print(objectives)

In [ ]:
#remove inputs
outputs = sim_samples.drop(features, axis=1)
#outputs

In [ ]:
#remove outputs and windows types shading
samples = sim_samples.drop(objectives, axis=1)
#samples

In [ ]:
for i,win in enumerate(win_arr):
    samples['Windows types'] = samples['Windows types'].replace(win,i+1)

In [ ]:
samples

## Train-test split

Next we split the data into a training set (80%) and a testing set (20%).

In [ ]:
train_in, test_in, train_out, test_out = train_test_split(
    samples, outputs, test_size=0.2
)

## Normalization of inputs

To ensure an equal weighting of inputs and outputs in the backpropagation algorithm fitting the neural network, we have to normalize the input values.
For example window-to-wall ratio is in the range of 0 to 1 while the $W/$m^2$ are in a range of 10 to 15.
Different options for normalization exist.
Here we bring all features (input variables) to have zero mean and a standarddeviation of 1.
Note that we fit the normalizer on training data only.

In [ ]:
scaler = StandardScaler()
train_in_scale = scaler.fit_transform(X=train_in)
test_in_scale = scaler.fit_transform(X=test_in)

scaler_out = StandardScaler()
train_out_scale = scaler_out.fit_transform(X=train_out)
test_out_scale = scaler_out.fit_transform(X=test_out)

# Gaussian Process


## Hyper-parameters

Before fitting the GP model we define the set of hyperparameters we want to optimize.
Here we use \textit{3} folds in the k-fold cross validation scheme.
We select a set of Kernel functions, which must fit the characteristics of a problem - details and examples may be found in the [Kernel cookbook](https://www.cs.toronto.edu/~duvenaud/cookbook/).
Note that the parameters of the Kernel itself are [optimized during each model fitting run](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html).

In [ ]:
hyperparameters = {
    "kernel": [
        None,
        1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)),
        1.0 * RationalQuadratic(length_scale=1.0, alpha=0.5),
        # ConstantKernel(0.1, (0.01, 10.0))*(DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0))**2),
        1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)),
    ]
}

folds = 3

## Model fitting

Here we fit the model using these hyperparameters.

In [ ]:
gp = GaussianProcessRegressor(normalize_y=True)

clf = GridSearchCV(gp, hyperparameters, cv=folds)

clf.fit(train_in_scale, train_out_scale)

best_gp = clf.best_estimator_ 

In [ ]:
prediction_gp = scaler_out.inverse_transform(best_gp.predict(test_in_scale))

# Neural Network

## Hyper-parameters

Before we start fitting the NN model we define the set of hyperparameters we want to analyse in our cross-validation to optimize the model.
Here, we select the number of layers of the network as well as the regularization parameter alpha as parameter value.
A larger number of layers and a lower value of the regularizer lead to higher variance of the network.
This may lead to overfitting.
The best selection may be found using an optimizer like Bayesian Optimization.
In this example we use a simple grid search.

In [ ]:
hyperparameters = {
    "hidden_layer_sizes": (
        (len(parameters) * 16,),
        (len(parameters) * 16, len(parameters) * 16),
    ),
    "alpha": [1, 10, 10 ** 3],
}

neural_net = MLPRegressor(max_iter=1000, early_stopping=False)
folds = 3

## Model fitting

Here, we use the NN model from ScikitLearn.


In [ ]:
clf = GridSearchCV(neural_net, hyperparameters, cv=folds)
clf.fit(train_in_scale, train_out_scale)

nn_model = clf.best_estimator_

In [ ]:
prediction_nn = scaler_out.inverse_transform(nn_model.predict(test_in_scale))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def build_model():
    model = keras.Sequential(
        [
            layers.Input(shape=(train_in_scale.shape[1], )),
            layers.Dense(32, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(3),
        ]
    )

    optimizer = tf.keras.optimizers.Adam(0.001)

    model.compile(loss="mse", optimizer=optimizer, metrics=["mae", "mse"])
    return model

tf_model = build_model()

tf_model.summary()

EPOCHS = 1000

history = tf_model.fit(
    train_in_scale,
    train_out_scale,
    epochs=EPOCHS,
    validation_split=0.2,
    verbose=0,
)

In [ ]:
prediction_tf = scaler_out.inverse_transform(tf_model.predict(test_in_scale))

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from math import sqrt


In [ ]:
mean_squared_error(prediction_gp,test_out.values,multioutput='raw_values',squared=False)/test_out.mean()*100

In [ ]:
mean_squared_error(prediction_nn,test_out.values,multioutput='raw_values')/test_out.mean()*100

In [ ]:
mean_squared_error(prediction_tf,test_out.values,multioutput='raw_values',squared=False)/test_out.mean()*100

Try with more samples 20 and 40

In [ ]:
#You can load the data from the files in the sim_samples_folder
# 20 samples generation took: 5 min 18 seconds
# 40 samples generation took: 19 min  7 seconds


## Surrogate Modelling Evaluator object
We can wrap the fitted model in a BESOS `Evaluator`.

In [ ]:
#Selection of windows parameters
win_arr=[1,2,3]
windowsTypes = CategoryParameter(win_arr)

windowsParameters = []

for fenestration in building.idfobjects["FenestrationSurface:Detailed"]:
     if fenestration.obj[2] == "Window":
        sel = FieldSelector(class_name ='FenestrationSurface:Detailed', object_name = fenestration.Name, field_name='Construction Name' )
        windowsParameters.append(Parameter(selector=sel, value_descriptors = windowsTypes, name='Windows types'))

In [ ]:
parameters = insulation_param + windowsParameters + [lights_param]  + ventilationAchParam + setpointParams

In [ ]:
#objectives and problem definition
objectives = ['Electricity:Facility','DistrictHeating:Facility','DistrictCooling:Facility']

problem=Problem(parameters, objectives)


In [ ]:
#definition of the evaluation function
def evaluation_func(ind, scaler=scaler):
    ind = scaler.transform(X=[ind])
    return (scaler_out.inverse_transform(nn_model.predict(ind))[0]).tolist()

evaluator = EvaluatorGeneric(evaluation_func, problem)


In [ ]:
srinputs = sampling.dist_sampler(sampling.lhs, problem, 500)
for i,win in enumerate(win_arr):
    srinputs['Windows types'] = srinputs['Windows types'].replace(win,i+1)
srinputs

In [ ]:
sroutputs = evaluator.df_apply(srinputs)
srresults = srinputs.join(sroutputs)
srresults.head()

## Exploration

In [ ]:
import plotly
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
import plotly.express as px
fig = px.parallel_coordinates(srresults,color="Electricity:Facility", dimensions=features+objectives,
                             color_continuous_scale=px.colors.diverging.Tealrose)
fig.show()

# Perform Building Optimization 

Using the best surrogate perfomr an otimization process, selct optimal values and save a new idf with the selected values. Evalute the goodnes of the surrogate simuation with EnergyPlus.

In [ ]:
from besos.optimizer import NSGAII

In [ ]:
#running NSGA-II optimizator 
results = NSGAII(evaluator, evaluations=5000, population_size=10000)



In [ ]:
optres = results.loc[results["pareto-optimal"] == True, :]  # Get only the optimal results

In [ ]:
#plotting results
plt.figure(figsize=(8, 6), dpi=80)
ax = plt.axes(projection='3d')



df = pd.DataFrame(optres, columns=features + objectives)

ax.plot3D(results["DistrictCooling:Facility"], results["DistrictHeating:Facility"], results["Electricity:Facility"], "x")  # Plot all results in the background as blue crosses
ax.plot3D(optres["DistrictCooling:Facility"], optres["DistrictHeating:Facility"], optres["Electricity:Facility"], "ro")  # Plot optimal results in red

ax.set_xlabel("Cooling demand")
ax.set_ylabel("Heating demand")
ax.set_zlabel("Electricity demand")



In [ ]:
optres

In [ ]:
_=pairplot(optres,x_vars=samples.columns, y_vars=objectives, kind="scatter")

In [ ]:
corr=optres.corr()

In [ ]:
unuseful = ['violation','pareto-optimal']


In [ ]:
corr.drop(objectives + unuseful, axis = 1, inplace = True)
corr.drop(features + unuseful, axis = 0 ,inplace = True)
corr

In [ ]:
plt.figure(figsize = (13,10))
_ = heatmap(corr,annot=True)

In [ ]:
df['Total'] = df['Electricity:Facility'] + df['DistrictHeating:Facility'] + df['DistrictCooling:Facility']
df['Dist'] = df.apply(lambda row : np.sqrt(pow(row["DistrictCooling:Facility"],2) + pow(row["DistrictHeating:Facility"],2) + pow(row["Electricity:Facility"],2)),axis=1)

df[df.Dist == df.Dist.min()]


In [ ]:
optimal_params=df.loc[df.Dist == df.Dist.min(),features].to_dict('records')[0]
optimal_params

In [ ]:
lights_selector.set(building,optimal_params['Lights Watts/Area'])
roof_ins.set(building,optimal_params['Roof Insulation'])
wall_ins.set(building,optimal_params['Wall Insulation'])
shade_setpoint_sel.set(building,optimal_params['Temp Setpoint shading'])
shade_setpoint2_sel.set(building,optimal_params['Rad Setpoint shading'])
ventACH_sel.set(building,optimal_params['Ventilation ACH'])
#selection of windows parameters
win_type = 'triple_glazing'
for fenestration in building.idfobjects["FenestrationSurface:Detailed"]:
     if fenestration.obj[2] == "Window":
        win_sel = FieldSelector(class_name ='FenestrationSurface:Detailed', 
                            object_name = fenestration.Name, 
                            field_name='Construction Name' )
        win_sel.set(building,win_type)

In [ ]:
building.saveas('lab_optimal.idf')